First Imports

In [1]:
import os
import json
import requests

API Key and URL

In [2]:
#os.environ['PL_API_KEY'] = '5dc3169d9ccd4f2f9a5d6617db785661'

PLANET_API_KEY = os.getenv('PL_API_KEY')
print(PLANET_API_KEY)

None


In [35]:
# Helper function to printformatted JSON using the json module
def printer(data):
    print(json.dumps(data, indent=2))

Requests

In [36]:
URL = 'https://api.planet.com/data/v1'

session = requests.Session()

res = session.get(URL, auth = (PLANET_API_KEY, '') )

In [37]:
res.status_code

200

In [38]:
res.text

'{"_links": {"_self": "https://api.planet.com/data/v1/", "asset-types": "https://api.planet.com/data/v1/asset-types/", "item-types": "https://api.planet.com/data/v1/item-types/", "spec": "https://api.planet.com/data/v1/spec"}}'

In [39]:
printer(res.json())

{
  "_links": {
    "_self": "https://api.planet.com/data/v1/",
    "asset-types": "https://api.planet.com/data/v1/asset-types/",
    "item-types": "https://api.planet.com/data/v1/item-types/",
    "spec": "https://api.planet.com/data/v1/spec"
  }
}


In [8]:
print(res.json()["_links"]["item-types"])

https://api.planet.com/data/v1/item-types/


Statistics endpoint

In [9]:
stats_url = "{}/stats".format(URL)
print(stats_url)

https://api.planet.com/data/v1/stats


**Filter Criteria:**
- Type (type) - The type of filter being used
- Configuration (config) - The configuration for this filter
- Field Name (field_name) - The field on which to filter

In [10]:
#Specify the sensors/satellites or "item types" to include in our results
item_types = ["PSScene3Band", "REOrthoTile"]

In [11]:
# Create filter object for all imagery captured between 2013-01-01 and present.
date_filter = {
    "type": "DateRangeFilter", # Type of filter -> Date Range
    "field_name": "acquired", # The field to filter on: "acquired" -> Date on which the "image was taken"
    "config": {
        "gte": "2013-01-01T00:00:00.000Z", # "gte" -> Greater than or equal to
    }
}


In [13]:
# Construct the request.
request = {
    "item_types" : item_types,
    "interval" : "year",
    "filter" : date_filter
}

# Send the POST request to the API stats endpoint
res = session.post(stats_url, json=request)

# Print response
printer(res.json())

{
  "utc_offset": "+0h",
  "interval": "year",
  "buckets": [
    {
      "count": 3095020,
      "start_time": "2013-01-01T00:00:00.000000Z"
    },
    {
      "count": 3596278,
      "start_time": "2014-01-01T00:00:00.000000Z"
    },
    {
      "count": 8491790,
      "start_time": "2015-01-01T00:00:00.000000Z"
    },
    {
      "count": 37679317,
      "start_time": "2016-01-01T00:00:00.000000Z"
    },
    {
      "count": 174726781,
      "start_time": "2017-01-01T00:00:00.000000Z"
    },
    {
      "count": 241833970,
      "start_time": "2018-01-01T00:00:00.000000Z"
    },
    {
      "count": 6172402,
      "start_time": "2019-01-01T00:00:00.000000Z"
    }
  ]
}


Practice Exercise

In [15]:
# Fill in this filter to complete the exercise! 
date_filter2 = {
    'type': 'DateRangeFilter',
    'field_name': 'acquired',
    'config': {
        'lte': "2013-01-01T00:00:00.000Z"
    }
}

request = {
    "item_types" : item_types,
    "interval" : "year",
    "filter" : date_filter
}

res = session.post(stats_url, json=request)
printer(res.json())

{
  "utc_offset": "+0h",
  "interval": "year",
  "buckets": [
    {
      "count": 3095020,
      "start_time": "2013-01-01T00:00:00.000000Z"
    },
    {
      "count": 3596278,
      "start_time": "2014-01-01T00:00:00.000000Z"
    },
    {
      "count": 8491790,
      "start_time": "2015-01-01T00:00:00.000000Z"
    },
    {
      "count": 37679317,
      "start_time": "2016-01-01T00:00:00.000000Z"
    },
    {
      "count": 174726781,
      "start_time": "2017-01-01T00:00:00.000000Z"
    },
    {
      "count": 241833970,
      "start_time": "2018-01-01T00:00:00.000000Z"
    },
    {
      "count": 6172407,
      "start_time": "2019-01-01T00:00:00.000000Z"
    }
  ]
}


**Field Filters**
- DateRangeFilter
- RangeFilter
- StringInFilter
- PermissionFilter
- GeometryFilter

**Logical Filters**
- NotFilter
- AndFilter
- OrFilter



**Camp Fire Dates:**

Began: 2018-11-08

Contained: 2018-11-25

End buring: 2018-??-??

In [3]:
camp_fire_filter = {  
   "type":"AndFilter",
   "config":[  
      {  
         "type":"DateRangeFilter",
         "field_name":"acquired",
         "config":{  
            "gte":"2018-11-01T00:00:00Z",
            "lte":"2018-12-01T00:00:00Z"
         }
      },
      {  
         "type":"GeometryFilter",
         "field_name":"geometry",
         "config":{"type":"Polygon",
                   "coordinates": [
          [
            [
              -122.11166381835936,
              39.977120098439634
            ],
            [
              -122.0745849609375,
              39.48814483559126
            ],
            [
              -121.09268188476562,
              39.54641191968671
            ],
            [
              -121.12976074218749,
              39.985538414809746
            ],
            [
              -122.11166381835936,
              39.977120098439634
            ]
          ]
        ]
                  }
      }
    ]
}

In [5]:
item_types = ['PSScene4Band']
request = {
    "item_types" : item_types,
    "interval" : "day",
    "filter" : camp_fire_filter
}

res = session.post(stats_url, json=request)
printer(res.json())

NameError: name 'session' is not defined

In [18]:
date_filter = {"type":"DateRangeFilter", 
               "field_name":"acquired", 
               "config":{"gte":"2018-11-01T00:00:00Z",
                         "lte":"2018-12-01T00:00:00Z"
                        }
              }

geo_filter = {"type":"GeometryFilter", 
              "field_name":"geometry", 
              "config":{"type":"Polygon",
                        "coordinates": [
                            [
                                        [
                                          -122.11166381835936,
                                          39.977120098439634
                                        ],
                                        [
                                          -122.0745849609375,
                                          39.48814483559126
                                        ],
                                        [
                                          -121.09268188476562,
                                          39.54641191968671
                                        ],
                                        [
                                          -121.12976074218749,
                                          39.985538414809746
                                        ],
                                        [
                                          -122.11166381835936,
                                          39.977120098439634
                                        ]
                                      ]
                        ]
                       }
             }


In [20]:
camp_filter = {"type":"AndFilter",
               "config":[date_filter, geo_filter]}

request = {
    "item_types" : item_types,
    "interval" : "month",
    "filter" : camp_filter 
}

res = session.post(stats_url, json=request)
printer(res.json())

{
  "utc_offset": "+0h",
  "interval": "month",
  "buckets": [
    {
      "count": 1508,
      "start_time": "2018-11-01T00:00:00.000000Z"
    }
  ]
}


In [21]:
ferguson_fire = {
    "type": "Polygon",
    "coordinates": [
        [
            [
              -120.39916992187499,
              37.58158917213053
            ],
            [
              -119.49554443359376,
              36.98280911070616
            ],
            [
              -118.67156982421875,
              37.58376576718623
            ],
            [
              -119.4873046875,
              38.20581359813473
            ],
            [
              -120.39916992187499,
              37.58158917213053
            ]
          ]
    ]
}

### Woolesly Fire

In [24]:
woolesly_fire = {"type": "Polygon", 
                 "coordinates": [
                 [
                  -119.036865234375,
                  33.99575015925125
                 ],
                 [
                  -118.54522705078126,
                  33.99802726234877
                 ],
                 [
                  -118.54248046874999,
                  34.3955786154917
                 ],
                 [
                  -119.04235839843749,
                  34.40464357107094
                 ],
                 [
                  -119.04373168945314,
                  33.99916579100914
                 ]
            ] 
        }

#### Quick Search

In [31]:
# Setup the quick search endpoint url
quick_url = "{}/quick-search".format(URL)

In [32]:
# Setup Item Types
item_types = ["PSScene4Band"]

# Setup GeoJSON for only imagery that intersects with 40N, 90W
geom = {
    "type": "Point",
    "coordinates": [
        -90,
         40
    ]
}

# Setup a geometry filter
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": "POINT(40 -90)"
}

# Setup the request
request = {
    "item_types" : item_types,
    "filter" : ferguson_fire
}

In [40]:
# Send the POST request to the API quick search endpoint
res = session.post(quick_url, json=request)

# Assign the response to a variable
geojson = res.json()

# Print the response
printer(geojson)

{
  "message": "Please enter your API key, or email and password.",
  "errors": []
}
